In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::env::VarError\")>]\n#endif\ntype std_env_VarError = class end"

nominal var_error = $'std_env_VarError'

/// ### os_environ
nominal os_environ = any

inl os_environ () : os_environ =
    open py_operators
    global "type IOsEnviron = abstract environ: x: unit -> obj"
    inl os : $'IOsEnviron' = py.import_all "os"
    !\($'"!os.environ"')

inl environ_get (key : string) (os_environ : os_environ) : string =
    open py_operators
    !\\(key, $'"!os_environ.get($0)"')

/// ## get_environment_variable
let get_environment_variable (var : string) : string =
    run_target function
        | Rust _ => fun () =>
            open rust_operators
            !\\(var, $'"std::env::var(&*$0)"')
            |> fun x => x : resultm.result' sm'.std_string var_error
            |> resultm.map' sm'.from_std_string
            |> resultm.unwrap_or' (join "")
        | Fsharp _ => fun () =>
            var
            |> $'System.Environment.GetEnvironmentVariable'
            |> optionm'.of_obj
            |> optionm'.unbox
            |> optionm'.default_value ""
        | TypeScript _ => fun () =>
            open ts_operators
            !\\(var, $'"process.env[$0] ?? \\\"\\\""')
        | Python _ => fun () =>
            os_environ ()
            |> environ_get var
        | target => fun () => failwith $'$"env.get_environment_variable / target: {!target} / var: {!var}"'

/// ## get_environment_variable_comptime
inl get_environment_variable_comptime (var : string) : string =
    run_target function
        | Rust (Native) => fun () =>
            open rust_operators
            !\($'"env\!(\\\"" + !var + "\\\")"')
            |> sm'.from_std_string
        | target => fun () => null ()

/// ## get_entry_assembly_name
let get_entry_assembly_name () : string =
    run_target function
        | Rust _ => fun () =>
            (join "CARGO_PKG_NAME") |> get_environment_variable
        | Fsharp _ => fun () =>
            $'System.Reflection.Assembly.GetEntryAssembly().GetName().Name'
        | target => fun () => failwith $'$"env.get_entry_assembly_name / target: {!target}"'

/// ## append_path
inl append_path (path : string) : option string =
    inl env_path = "PATH" |> get_environment_variable
    if env_path = ""
    then None
    else
        inl env_sep =
            if platform.is_windows ()
            then ";"
            else ":"
        Some $'$"{!path}{!env_sep}{!env_path}"'